In [11]:
import numpy as np
import pandas as pd

In [13]:
df=pd.read_csv("covid_toy.csv")

In [15]:
df.head(3)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No


In [17]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [19]:
from sklearn.impute import SimpleImputer
si=SimpleImputer()
df["fever"]=si.fit_transform(df[["fever"]])

In [21]:
df=pd.get_dummies(df,columns=["gender","city"])

In [29]:
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()
df["cough"]=oe.fit_transform(df[["cough"]])

In [31]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["has_covid"]=le.fit_transform(df["has_covid"])

In [33]:
df.head(3)

,age,fever,cough,has_covid,gender_Female,gender_Male,city_Bangalore,city_Delhi,city_Kolkata,city_Mumbai
0,60,103.0,0.0,0,False,True,False,False,True,False
1,27,100.0,0.0,1,False,True,False,True,False,False
2,42,101.0,0.0,0,False,True,False,True,False,False


In [35]:
df=df.astype(int)

In [37]:
df.head(2)

,age,fever,cough,has_covid,gender_Female,gender_Male,city_Bangalore,city_Delhi,city_Kolkata,city_Mumbai
0,60,103,0,0,0,1,0,0,1,0
1,27,100,0,1,0,1,0,1,0,0


In [39]:
df["has_covid"].value_counts()

has_covid
0    55
1    45
Name: count, dtype: int64

In [41]:
x=df.drop(columns="has_covid")
y=df["has_covid"]

In [43]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [51]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
pipe=Pipeline([
    ("scaler",StandardScaler()),
    ("pca",PCA(n_components=2)),
    ("classifier",RandomForestClassifier(n_estimators=100,random_state=42))
])

In [53]:
pipe

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [55]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [57]:
y_pred=pipe.predict(x_test)

In [59]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.45

In [87]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Lasso
parameterdict={"alpha":[0.1,1.0,10.0,100.0]}
lasso=Lasso()
random_search=RandomizedSearchCV( lasso, parameterdict, n_iter=4, cv=5)
random_search.fit(x_train,y_train)

RandomizedSearchCV(cv=5, estimator=Lasso(), n_iter=4,
                   param_distributions={'alpha': [0.1, 1.0, 10.0, 100.0]})

In [89]:
y_pred_new=random_search.predict(x_test)

In [91]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred_new)

-0.06868131868131888